# Maximum Number of Points From Grid Queries

You are given an m x n integer matrix grid and an array queries of size k.

Find an array answer of size k such that for each integer queries[i] you start in the top left cell of the matrix and repeat the following process:

If queries[i] is strictly greater than the value of the current cell that you are in, then you get one point if it is your first time visiting this cell, and you can move to any adjacent cell in all 4 directions: up, down, left, and right.
Otherwise, you do not get any points, and you end this process.
After the process, answer[i] is the maximum number of points you can get. Note that for each query you are allowed to visit the same cell multiple times.

Return the resulting array answer.

In [ ]:
class Solution:
    def maxPoints(self, grid: List[List[int]], queries: List[int]) -> List[int]:
        m, n = len(grid), len(grid[0])
        # Prepare sorted queries with original indices
        sorted_queries = sorted([(q, i) for i, q in enumerate(queries)])
        # Prepare sorted list of grid cells: (value, row, col)
        cells = sorted([(grid[i][j], i, j) for i in range(m) for j in range(n)])
        
        # Initialize union-find structure and activated matrix
        uf = UnionFind(m * n)
        activated = [[False] * n for _ in range(m)]
        
        ans = [0] * len(queries)
        cell_idx = 0
        
        # Define directions: up, down, left, right
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        
        for query, idx in sorted_queries:
            # Activate grid cells with value < query
            while cell_idx < len(cells) and cells[cell_idx][0] < query:
                _, i, j = cells[cell_idx]
                activated[i][j] = True
                pos = i * n + j  # flatten index
                
                # Union with any adjacent activated cells
                for di, dj in directions:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < m and 0 <= nj < n and activated[ni][nj]:
                        uf.union(pos, ni * n + nj)
                cell_idx += 1
            
            # If the top-left cell is activated, answer is the size of its component
            if activated[0][0]:
                ans[idx] = uf.size(0)  # Assuming uf.size returns component size for the root of index 0
            else:
                ans[idx] = 0
        
        return ans


class UnionFind:
    def __init__(self, n):
        # Parent array where each node is initially its own parent.
        self.parent = list(range(n))
        # Rank array to help keep the tree flat.
        self.rank = [0] * n
        # Size array to track the size of each component.
        self.size_arr = [1] * n

    def find(self, x):
        # Path compression heuristic.
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        # Find the roots of x and y.
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX == rootY:
            return

        # Union by rank heuristic.
        if self.rank[rootX] < self.rank[rootY]:
            self.parent[rootX] = rootY
            self.size_arr[rootY] += self.size_arr[rootX]
        elif self.rank[rootX] > self.rank[rootY]:
            self.parent[rootY] = rootX
            self.size_arr[rootX] += self.size_arr[rootY]
        else:
            self.parent[rootY] = rootX
            self.rank[rootX] += 1
            self.size_arr[rootX] += self.size_arr[rootY]

    def size(self, x):
        # Return the size of the component that contains x.
        return self.size_arr[self.find(x)]